# 3. Machine Learning for Classification

We'll use logistic regression to predict churn

## 3.1 Churn prediction project

- Dataset: [http://www.kaggle.com/blastchar/telco-customer-churn](http://www.kaggle.com/blastchar/telco-customer-churn)

## 3.2 Data preparation

- Download the dataset, read it with pandas
- Look at the data
- Make column names and values look uniform
- Check if all columns read correctly
- Check if the churn variable needs any preparation

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [1]:
# !unzip ./data/archive.zip -d ./data

Archive:  ./data/archive.zip
  inflating: ./data/WA_Fn-UseC_-Telco-Customer-Churn.csv  


In [4]:
df = pd.read_csv("../data/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract PaperlessBilling  \
0          No          No              No  Month-to-month              Yes   
1          No          No              No        One year               No   
2          No          No              No  Month-to-month              Yes   
3         Yes          No              No        One year               No   
4          No          No              No  Month-to-month              Yes   

               PaymentMethod MonthlyCharges  TotalCharges Churn  
0           Electronic check          29.85         29.85    No  
1               Mailed check          56.95        1889.5    No  
2               Mailed check          53.85        108.15   Yes  
3  Bank transfer (automatic)          42.30       1840.75    No  
4           Electronic check          70.70        151.65   Yes  

[5 rows x 21 columns]

In [5]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

categorical_columns = list(df.dtypes[df.dtypes == "object"].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(" ", "_")

In [7]:
df.head().T

0             1               2  \
customerid              7590-vhveg    5575-gnvde      3668-qpybk   
gender                      female          male            male   
seniorcitizen                    0             0               0   
partner                        yes            no              no   
dependents                      no            no              no   
tenure                           1            34               2   
phoneservice                    no           yes             yes   
multiplelines     no_phone_service            no              no   
internetservice                dsl           dsl             dsl   
onlinesecurity                  no           yes             yes   
onlinebackup                   yes            no             yes   
deviceprotection                no           yes              no   
techsupport                     no            no              no   
streamingtv                     no            no              no   
streamingmovies                 no            no              no   
contract            month-to-month      one_year  month-to-month   
paperlessbilling               yes            no             yes   
paymentmethod     electronic_check  mailed_check    mailed_check   
monthlycharges               29.85         56.95           53.85   
totalcharges                 29.85        1889.5          108.15   
churn                           no            no             yes   

                                          3                 4  
customerid                       7795-cfocw        9237-hqitu  
gender                                 male            female  
seniorcitizen                             0                 0  
partner                                  no                no  
dependents                               no                no  
tenure                                   45                 2  
phoneservice                             no               yes  
multiplelines              no_phone_service                no  
internetservice                         dsl       fiber_optic  
onlinesecurity                          yes                no  
onlinebackup                             no                no  
deviceprotection                        yes                no  
techsupport                             yes                no  
streamingtv                              no                no  
streamingmovies                          no                no  
contract                           one_year    month-to-month  
paperlessbilling                         no               yes  
paymentmethod     bank_transfer_(automatic)  electronic_check  
monthlycharges                         42.3              70.7  
totalcharges                        1840.75            151.65  
churn                                    no               yes

In [8]:
df.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

`totalcharges` is read as an object, we need to convert it to a number.
`seniorcitizen` is read as an integer, we need to convert it to a categorical variable.

In [9]:
tc = pd.to_numeric(df.totalcharges, errors="coerce")

In [10]:
df.totalcharges = pd.to_numeric(df.totalcharges, errors="coerce")

In [11]:
df.totalcharges = df.totalcharges.fillna(0)

In [12]:
df.churn = (df.churn == "yes").astype(int)

## 3.3 Setting up the validation framework

- Perform the train/validation/test split with Scikit-Learn

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [15]:
len(df_full_train), len(df_test)

(5634, 1409)

In [16]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [17]:
len(df_train), len(df_val), len(df_test)

(4225, 1409, 1409)

In [18]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [19]:
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

In [20]:
del df_train["churn"]
del df_val["churn"]
del df_test["churn"]

## 3.4 EDA

- Check missing values
- Look at the target variable (churn)
- Look at numerical and categorical variables

In [21]:
df_full_train = df_full_train.reset_index(drop=True)

In [22]:
df_full_train.churn.value_counts(normalize=True)

churn
0    0.730032
1    0.269968
Name: proportion, dtype: float64

Target variable is imbalanced, we need to keep that in mind. `churn = 1` is called churn rate, `churn = 0` is called retention rate.

In [23]:
# In case of binary classification
global_churn_rate = df_full_train.churn.mean()
round(global_churn_rate, 2)

0.27

In [24]:
numerical = ["tenure", "monthlycharges", "totalcharges"]

In [25]:
df_full_train.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [26]:
categorical = [
    "gender",
    "seniorcitizen",
    "partner",
    "dependents",
    "phoneservice",
    "multiplelines",
    "internetservice",
    "onlinesecurity",
    "onlinebackup",
    "deviceprotection",
    "techsupport",
    "streamingtv",
    "streamingmovies",
    "contract",
    "paperlessbilling",
    "paymentmethod",
]

In [27]:
df_full_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

## 3.5 Feature importance: Churn rate and risk ratio

Feature importance analysis (part of EDA) - identifying which affect our target variable
- Churn rate
- Risk ratio
- Mutual information - later

### Churn rate

In [28]:
df_full_train.head()

customerid  gender  seniorcitizen partner dependents  tenure phoneservice  \
0  5442-pptjy    male              0     yes        yes      12          yes   
1  6261-rcvns  female              0      no         no      42          yes   
2  2176-osjuv    male              0     yes         no      71          yes   
3  6161-erdgd    male              0     yes        yes      71          yes   
4  2364-ufrom    male              0      no         no      30          yes   

  multiplelines internetservice       onlinesecurity  ...  \
0            no              no  no_internet_service  ...   
1            no             dsl                  yes  ...   
2           yes             dsl                  yes  ...   
3           yes             dsl                  yes  ...   
4            no             dsl                  yes  ...   

      deviceprotection          techsupport          streamingtv  \
0  no_internet_service  no_internet_service  no_internet_service   
1                  yes                  yes                   no   
2                   no                  yes                   no   
3                  yes                  yes                  yes   
4                   no                  yes                  yes   

       streamingmovies  contract paperlessbilling              paymentmethod  \
0  no_internet_service  two_year               no               mailed_check   
1                  yes  one_year               no    credit_card_(automatic)   
2                   no  two_year               no  bank_transfer_(automatic)   
3                  yes  one_year               no           electronic_check   
4                   no  one_year               no           electronic_check   

  monthlycharges  totalcharges  churn  
0          19.70        258.35      0  
1          73.90       3160.55      1  
2          65.15       4681.75      0  
3          85.45       6300.85      0  
4          70.40       2044.75      0  

[5 rows x 21 columns]

In [29]:
churn_female = df_full_train[df_full_train.gender == "female"].churn.mean()
churn_female.round(2)

0.28

In [30]:
churn_male = df_full_train[df_full_train.gender == "male"].churn.mean()
churn_male.round(2)

0.26

Churn rate for female and male don't differ much from global churn rate.

Let's look at other features.

In [31]:
df_full_train.partner.value_counts()

partner
no     2932
yes    2702
Name: count, dtype: int64

In [32]:
churn_partner = df_full_train[df_full_train.partner == "yes"].churn.mean()
churn_partner.round(2)

0.21

In [33]:
global_churn_rate - churn_partner

0.06493474245795922

In [34]:
churn_no_partner = df_full_train[df_full_train.partner == "no"].churn.mean()
churn_no_partner.round(2)

0.33

In [35]:
global_churn_rate - churn_no_partner

-0.05984095297455855

Difference in churn rate for `partner` is much bigger, compared to male or female.

1. Differences

    global - group

    < 0      > 0

    more likely to churn - less likely to churn

2. Risk ratio

    Risk = group / global

    \> 1 - more likely to churn

    < 1 - less likely to churn

<img src="../images/churn_rate_differences.png" alt="Churn Rate and Risk Ratio" style="width:300px;height:auto;">

### Risk ratio

In [36]:
churn_no_partner / global_churn_rate

1.2216593879412643

In [37]:
churn_partner / global_churn_rate

0.7594724924338315

```sql
SELECT
    gender,
    AVG(churn),
    AVG(churn) - global_churn_rate AS diff,
    AVG(churn) / global_churn_rate AS risk
FROM
    data
GROUP BY
    gender;
```

In [38]:
for c in categorical:
    print(c)
    df_group = df_full_train.groupby(c).churn.agg(["mean", "count"])
    df_group["diff"] = df_group["mean"] - global_churn_rate
    df_group["risk"] = df_group["mean"] / global_churn_rate
    display(df_group)
    print()

gender


mean  count      diff      risk
gender                                     
female  0.276824   2796  0.006856  1.025396
male    0.263214   2838 -0.006755  0.974980


seniorcitizen


mean  count      diff      risk
seniorcitizen                                     
0              0.242270   4722 -0.027698  0.897403
1              0.413377    912  0.143409  1.531208


partner


...

mean  count      diff      risk
contract                                           
month-to-month  0.431701   3104  0.161733  1.599082
one_year        0.120573   1186 -0.149395  0.446621
two_year        0.028274   1344 -0.241694  0.104730


paperlessbilling


mean  count      diff      risk
paperlessbilling                                     
no                0.172071   2313 -0.097897  0.637375
yes               0.338151   3321  0.068183  1.252560


paymentmethod


mean  count      diff      risk
paymentmethod                                                 
bank_transfer_(automatic)  0.168171   1219 -0.101797  0.622928
credit_card_(automatic)    0.164339   1217 -0.105630  0.608733
electronic_check           0.455890   1893  0.185922  1.688682
mailed_check               0.193870   1305 -0.076098  0.718121

## 3.6 Feature importance: Mutual information

Mutual information - concept from information theory, it tells us how much we can learn about one variable if we know the value of another
- [https://en.wikipedia.org/wiki/Mutual_information](https://en.wikipedia.org/wiki/Mutual_information)

In [39]:
from sklearn.metrics import mutual_info_score

In [40]:
mutual_info_score(df_full_train.churn, df_full_train.contract)

0.0983203874041556

In [41]:
mutual_info_score(df_full_train.gender, df_full_train.churn)

0.0001174846211139946

In [42]:
mutual_info_score(df_full_train.contract, df_full_train.churn)

0.0983203874041556

In [43]:
mutual_info_score(df_full_train.partner, df_full_train.churn)

0.009967689095399745

In [44]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.churn)

In [45]:
mi = (
    df_full_train[categorical]
    .apply(mutual_info_churn_score)
    .sort_values(ascending=False)
)
mi

contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64

## 3.7 Feature importance: Correlation

How about numerical columns?
- Correlation coefficient - [https://en.wikipedia.org/wiki/Pearson_correlation_coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient)

In [46]:
df_full_train[numerical].corrwith(df_full_train.churn)

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64

In [47]:
df_full_train[df_full_train.tenure <= 2].churn.mean()

0.5953420669577875

In [48]:
df_full_train[(df_full_train.tenure > 2) & (df_full_train.tenure <= 12)].churn.mean()


0.3994413407821229

In [49]:
df_full_train[df_full_train.tenure > 12].churn.mean()


0.17634908339788277

In [50]:
df_full_train[df_full_train.monthlycharges <= 20].churn.mean()


0.08795411089866156

In [51]:
df_full_train[
    (df_full_train.monthlycharges > 20) & (df_full_train.monthlycharges <= 50)
].churn.mean()


0.18340943683409436

In [52]:
df_full_train[df_full_train.monthlycharges > 50].churn.mean()

0.32499341585462205

<img src="../images/churn_rate_correlation.png" alt="Correlation" style="width:300px;height:auto;">

Tenure: negative correlation

Monthly charges: positive correlation

## 3.8 One-hot encoding

- Use Scikit-Learn to encode categorical features

In [53]:
from sklearn.feature_extraction import DictVectorizer

In [54]:
dv = DictVectorizer(sparse=False)

train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)

## 3.9 Logistic regression

- Binary classification
- Linear vs logistic regression

In [55]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [56]:
z = np.linspace(-7, 7, 51)

In [57]:
sigmoid(z)

array([9.11051194e-04, 1.20508423e-03, 1.59386223e-03, 2.10780106e-03,
       2.78699622e-03, 3.68423990e-03, 4.86893124e-03, 6.43210847e-03,
       8.49286285e-03, 1.12064063e-02, 1.47740317e-02, 1.94550846e-02,
       2.55807883e-02, 3.35692233e-02, 4.39398154e-02, 5.73241759e-02,
       7.44679452e-02, 9.62155417e-02, 1.23467048e-01, 1.57095469e-01,
       1.97816111e-01, 2.46011284e-01, 3.01534784e-01, 3.63547460e-01,
       4.30453776e-01, 5.00000000e-01, 5.69546224e-01, 6.36452540e-01,
       6.98465216e-01, 7.53988716e-01, 8.02183889e-01, 8.42904531e-01,
       8.76532952e-01, 9.03784458e-01, 9.25532055e-01, 9.42675824e-01,
       9.56060185e-01, 9.66430777e-01, 9.74419212e-01, 9.80544915e-01,
       9.85225968e-01, 9.88793594e-01, 9.91507137e-01, 9.93567892e-01,
       9.95131069e-01, 9.96315760e-01, 9.97213004e-01, 9.97892199e-01,
       9.98406138e-01, 9.98794916e-01, 9.99088949e-01])

In [58]:
plt.plot(z, sigmoid(z))

In [59]:
# Remainder: Linear regression

# def linear_regression(xi):
#     result = w0

#     for j in range(len(w)):
#         result = result + xi[j] * w[j]

#     return result

In [60]:
# Basic logistic regression implementation

# def logistic_regression(xi):
#     score = w0

#     for j in range(len(w)):
#         score = score + xi[j] * w[j]

#     result = sigmoid(score)
#     return result

## 3.10 Training logistic regression with Scikit-Learn

- Train a model with Scikit-Learn
- Apply it to the validation dataset
- Calculate the accuracy

In [61]:
from sklearn.linear_model import LogisticRegression

In [62]:
model = LogisticRegression(max_iter=100000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=100000)

In [63]:
model.intercept_[0]

-0.05005078279271729

In [64]:
model.coef_[0].round(3)

array([ 0.677,  0.028, -0.673,  0.05 , -0.018,  0.11 , -0.157,  0.078,
        0.035, -0.003, -0.491,  0.68 , -0.157, -0.017, -0.191,  0.072,
        0.151,  0.114, -0.157,  0.075,  0.282, -0.157, -0.093, -0.166,
        0.198, -0.047,  0.079, -0.055, -0.006,  0.104, -0.011,  0.072,
       -0.04 ,  0.194, -0.094, -0.157,  0.283, -0.055, -0.157,  0.243,
        0.232, -0.157, -0.043, -0.068,  0.   ])

In [65]:
y_pred = model.predict_proba(X_val)[:, 1]

In [66]:
churn_decision = y_pred >= 0.5

In [67]:
df_val[churn_decision].customerid

3       8433-wxgna
8       3440-jpscl
12      7228-omtpn
19      6711-fldfb
24      2612-ranwt
           ...    
1397    5976-jcjrh
1398    2034-cgrhz
1399    5276-kqwhg
1407    6521-yytyi
1408    3049-solay
Name: customerid, Length: 314, dtype: object

In [68]:
(y_val == churn_decision).mean()

0.8041163946061036

In [69]:
df_pred = pd.DataFrame()
df_pred["probability"] = y_pred
df_pred["prediction"] = churn_decision.astype(int)
df_pred["actual"] = y_val

In [70]:
df_pred["correct"] = df_pred.prediction == df_pred.actual

In [71]:
df_pred.correct.mean()

0.8041163946061036

## 3.11 Model interpretation

- Look at the coefficients
- Train a smaller model with fewer features

In [72]:
a = [1, 2, 3, 4]
b = "abcd"

In [73]:
dict(zip(a, b))

{1: 'a', 2: 'b', 3: 'c', 4: 'd'}

In [74]:
dict(zip(dv.get_feature_names_out(), model.coef_[0].round(3)))

{'contract=month-to-month': 0.677,
 'contract=one_year': 0.028,
 'contract=two_year': -0.673,
 'dependents=no': 0.05,
 'dependents=yes': -0.018,
 'deviceprotection=no': 0.11,
 'deviceprotection=no_internet_service': -0.157,
 'deviceprotection=yes': 0.078,
 'gender=female': 0.035,
 'gender=male': -0.003,
 'internetservice=dsl': -0.491,
 'internetservice=fiber_optic': 0.68,
 'internetservice=no': -0.157,
 'monthlycharges': -0.017,
 'multiplelines=no': -0.191,
 'multiplelines=no_phone_service': 0.072,
 'multiplelines=yes': 0.151,
 'onlinebackup=no': 0.114,
 'onlinebackup=no_internet_service': -0.157,
 'onlinebackup=yes': 0.075,
 'onlinesecurity=no': 0.282,
 'onlinesecurity=no_internet_service': -0.157,
 'onlinesecurity=yes': -0.093,
 'paperlessbilling=no': -0.166,
 'paperlessbilling=yes': 0.198,
 'partner=no': -0.047,
 'partner=yes': 0.079,
 'paymentmethod=bank_transfer_(automatic)': -0.055,
 'paymentmethod=credit_card_(automatic)': -0.006,
 'paymentmethod=electronic_check': 0.104,
 'paym

In [75]:
small = ["contract", "tenure", "monthlycharges"]

In [76]:
df_train[small].iloc[:10].to_dict(orient="records")

[{'contract': 'two_year', 'tenure': 72, 'monthlycharges': 115.5},
 {'contract': 'month-to-month', 'tenure': 10, 'monthlycharges': 95.25},
 {'contract': 'month-to-month', 'tenure': 5, 'monthlycharges': 75.55},
 {'contract': 'month-to-month', 'tenure': 5, 'monthlycharges': 80.85},
 {'contract': 'two_year', 'tenure': 18, 'monthlycharges': 20.1},
 {'contract': 'month-to-month', 'tenure': 4, 'monthlycharges': 30.5},
 {'contract': 'month-to-month', 'tenure': 1, 'monthlycharges': 75.1},
 {'contract': 'month-to-month', 'tenure': 1, 'monthlycharges': 70.3},
 {'contract': 'two_year', 'tenure': 72, 'monthlycharges': 19.75},
 {'contract': 'month-to-month', 'tenure': 6, 'monthlycharges': 109.9}]

In [77]:
dicts_train_small = df_train[small].to_dict(orient="records")
dicts_val_small = df_val[small].to_dict(orient="records")
# dicts_test_small = df_test[small].to_dict(orient='records')

In [78]:
dv_small = DictVectorizer(sparse=False)
dv_small.fit(dicts_train_small)

DictVectorizer(sparse=False)

In [79]:
dv_small.get_feature_names_out()

array(['contract=month-to-month', 'contract=one_year',
       'contract=two_year', 'monthlycharges', 'tenure'], dtype=object)

In [80]:
X_train_small = dv_small.transform(dicts_train_small)

In [81]:
model_small = LogisticRegression(max_iter=100000)
model_small.fit(X_train_small, y_train)

LogisticRegression(max_iter=100000)

In [82]:
w0 = model_small.intercept_[0]
w0

-2.477957595779287

In [83]:
w = model_small.coef_[0]
w.round(3)

array([ 0.971, -0.024, -0.948,  0.027, -0.036])

In [84]:
dict(zip(dv_small.get_feature_names_out(), w.round(3)))

{'contract=month-to-month': 0.971,
 'contract=one_year': -0.024,
 'contract=two_year': -0.948,
 'monthlycharges': 0.027,
 'tenure': -0.036}

In [85]:
-2.47 + 0.97 + 50 * 0.027 + 5 * (-0.036)

-0.3300000000000001

In [86]:
sigmoid(_) # `_` is a magic variable that stores the last output

0.41824062315816374

## 3.12 Using the model

In [87]:
dicts_full_train = df_full_train[categorical + numerical].to_dict(orient="records")

In [88]:
dv = DictVectorizer(sparse=False)
X_full_train = dv.fit_transform(dicts_full_train)

In [89]:
y_full_train = df_full_train.churn.values

In [90]:
model = LogisticRegression(max_iter=100000)
model.fit(X_full_train, y_full_train)

LogisticRegression(max_iter=100000)

In [91]:
dicts_test = df_test[categorical + numerical].to_dict(orient="records")

In [92]:
X_test = dv.transform(dicts_test)

In [93]:
y_pred = model.predict_proba(X_test)[:, 1]

In [94]:
churn_decision = (y_pred >= 0.5)

In [95]:
(churn_decision == y_test).mean()

0.8126330731014905

Only 1% difference in accuracy between the full and reduced models.

In [96]:
customer = dicts_test[-1]
customer

{'gender': 'female',
 'seniorcitizen': 0,
 'partner': 'yes',
 'dependents': 'yes',
 'phoneservice': 'yes',
 'multiplelines': 'yes',
 'internetservice': 'fiber_optic',
 'onlinesecurity': 'yes',
 'onlinebackup': 'no',
 'deviceprotection': 'yes',
 'techsupport': 'no',
 'streamingtv': 'yes',
 'streamingmovies': 'yes',
 'contract': 'month-to-month',
 'paperlessbilling': 'yes',
 'paymentmethod': 'electronic_check',
 'tenure': 17,
 'monthlycharges': 104.2,
 'totalcharges': 1743.5}

In [97]:
X_small = dv.transform([customer])

In [98]:
model.predict_proba(X_small)[0, 1]

0.640259093091188

In [99]:
y_test[-1]

1

Continued in [04_metrics.ipynb](04_metrics.ipynb) and [05_serve_model.ipynb](05_serve_model.ipynb).


## 3.13 Summary

- Feature importance - risk, mutual information, correlation
- One-hot encoding can be implemented with `DictVectorizer`
- Logistic regression - linear model like linear regression
- Output of log reg - probability
- Interpretation of weights is similar to linear regression

## 3.14 Explore more

More things
- Try to exclude least useful features

In [100]:
df_train.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges'],
      dtype='object')

In [101]:
least_useful_features_excluded = [
    "customerid",
    "seniorcitizen",
    "partner",
    "dependents",
    "tenure",
    "internetservice",
    "onlinesecurity",
    "onlinebackup",
    "deviceprotection",
    "techsupport",
    "streamingtv",
    "streamingmovies",
    "contract",
    "paperlessbilling",
    "paymentmethod",
    "monthlycharges",
    "totalcharges",
]

In [102]:
X_train_lufe = dv.fit_transform(df_train[least_useful_features_excluded].to_dict(orient="records"))
X_val_lufe = dv.transform(df_val[least_useful_features_excluded].to_dict(orient="records"))

In [103]:
model_lufe = LogisticRegression(max_iter=100000)
model_lufe.fit(X_train_lufe, y_train)

LogisticRegression(max_iter=100000)

In [104]:
y_pred_lufe = model_lufe.predict_proba(X_val_lufe)[:, 1]

In [105]:
churn_decision_lufe = y_pred_lufe >= 0.5

In [106]:
df_val[churn_decision_lufe].customerid

3       8433-wxgna
8       3440-jpscl
11      2637-fkfsy
12      7228-omtpn
19      6711-fldfb
           ...    
1397    5976-jcjrh
1398    2034-cgrhz
1399    5276-kqwhg
1407    6521-yytyi
1408    3049-solay
Name: customerid, Length: 319, dtype: object

In [107]:
(y_val == churn_decision_lufe).mean()

0.8005677785663591

In [108]:
df_pred_lufe = pd.DataFrame()
df_pred_lufe["probability"] = y_pred_lufe
df_pred_lufe["prediction"] = churn_decision_lufe.astype(int)
df_pred_lufe["actual"] = y_val

In [111]:
df_pred_lufe["correct"] = df_pred_lufe.prediction == df_pred_lufe.actual
df_pred_lufe.correct.mean()

0.8005677785663591


Use scikit-learn in project of last week
- Re-implement train/val/test split using scikit-learn in the project from last week
- Also, instead of our own linear regression, use `LinearRegression` (not regularized) and `RidgeRegression` (regularized). Find the best regularization parameter for Ridge.

Other projects
- Lead scoring - [https://www.kaggle.com/ashydv/leads-dataset](https://www.kaggle.com/ashydv/leads-dataset)
- Default prediction - [https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients)

Type *Markdown* and LaTeX: $\alpha^2$